# Data Prep for Find and Link Demo

Prepare publication and patient data for use in find/link entities demo accompanying blog post. 

This notebook is for demo purposes. You don't need to run this notebook, as we have already prepared the data for use in s3://aws-neptune-customer-samples/neptune-ent-res. If you're like most readers, you will want to focus on querying, rather than preparing, this data.

If you wish to prepare the data yourself, copy this notebook to a notebook instance. It uses AWS CLI to download a few source files. To ensure you have all the right dependencies, run from a SageMaker notebook instance. 

The files it prepares are saved to disk on the instance, so ensure you have enough space.

## Library to create bulk-loadable files for labeled property graph (LPG) or resource description framework (RDF)

See more about Neptune bulk loader here: https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load.html.

In [ ]:
'''
Library to manage CSV and RDF files for bulk load
'''

import pandas as pd
import urllib.parse
import csv
from rdflib import Graph, Literal, RDF, RDFS, URIRef, XSD, OWL, BNode

NS = "http://example.org/demo" # default, but override as you see fit

MAX_RDF_ROWS = 1000000

def csv_open(fname, headers):
    ff = []
    ff.append(open(fname, "w"))
    ff.append(csv.writer(ff[0]))
    csv_write(ff, headers)
    return ff

def csv_close(ff):
    ff[0].close()

def csv_write(ff, arr):
    ff[1].writerow("" if pd.isna(a) or a=='-' else a for a in arr)
    
def rdf_open(fname):
    toks = fname.rsplit(".")
    ff = [Graph(), fname, toks[0], toks[1], 1]
    return ff
    
def make_uri(name):
    return URIRef(f"{NS}/{name}")

def rdf_write(ff, s, p, o):
    # rdblib does not flush, so break it up as it gets larger
    if (ff[4] % MAX_RDF_ROWS) == 0:
        old_count = ff[4]
        new_fname = f"{ff[2]}{old_count // MAX_RDF_ROWS}.{ff[3]}"
        print("RDF split " + new_fname + " on " + str(old_count))
        rdf_close(ff)
        ff[0] = Graph()
        ff[1]=new_fname
        ff[4]=old_count

    ff[4] = ff[4] + 1
    ff[0].add((s, p, o))

def rdf_close(ff):
    ff[0].serialize(destination = ff[1], format='ntriples')
    ff[0] = None
    


## Patient Data

https://synthea.mitre.org/downloads

Jason Walonoski, Mark Kramer, Joseph Nichols, Andre Quina, Chris Moesel, Dylan Hall, Carlton Duffett, Kudakwashe Dube, Thomas Gallagher, Scott McLachlan, Synthea: An approach, method, and software mechanism for generating synthetic patients and the synthetic electronic health care record, Journal of the American Medical Informatics Association, Volume 25, Issue 3, March 2018, Pages 230–238, https://doi.org/10.1093/jamia/ocx079

### First create folder structure.

In [ ]:
%%bash

rm -rf patients
mkdir -p patients/source/downloads
mkdir -p patients/source/generated
mkdir -p patients/bulk/pg/nodes
mkdir -p patients/bulk/pg/edges


### Download the synthea patient set.

In [ ]:
%%bash
cd patients/source/downloads
wget https://mitre.box.com/shared/static/9iglv8kbs1pfi7z8phjl9sbpjk08spze.zip
unzip 9iglv8kbs1pfi7z8phjl9sbpjk08spze.zip
rm 9iglv8kbs1pfi7z8phjl9sbpjk08spze.zip

### Add a few duplicate patients. Will have similar name and location, but none of the encounters or payers.

In [ ]:
import os
import re
import pandas as pd
import random

# This cell modifies source patients.csv as follows:
# 1. It strips digits from the names. The source has names like Haley12. It strips to just Haley. 
#    This helps when doing full-text search of names.
# 2. For a random patient it adds a new matching patient with same zip, SIMILAR first name and:
# a) Same last name .
# b) Similar last name.
# 3. Randomly adds patients with same last name but different/dissimilar first name to the same zip as existing patient.

def strip_digits(col, s):
    if col in ['FIRST', 'LAST', 'MAIDEN']:
        if s == 'nan':
            return ''
        return re.sub(r'\d+', '', s)
    else:
        return s
        
def has_val(cell):
    return not(cell != cell) and len(str(cell)) > 0 and str(cell) != 'nan'
        
patients_df = pd.read_csv('patients/source/downloads/10k_synthea_covid19_csv/patients.csv')
new_rows = []
cols = patients_df.columns
zips_ldiff = []
for index, row in patients_df.iterrows():
    id = row['Id'] 
    this_row = {}
    for c in cols:
        this_row[str(c)] = strip_digits(c, str(row[c]))
    new_rows.append(this_row)
    
    # random row
    if has_val(this_row['ZIP']) and random.randint(1, 30) < 7:
        mod_row = this_row.copy()
        
        #new ID
        mod_row['Id'] = "x" + id
        
        # munge the first name
        fn = this_row['FIRST']
        mod_row['FIRST'] = fn[:1] + 'x' + fn[1+1:]
        
        # munge the last name
        lmod = False
        if random.randint(1, 20) < 7:
            lmod = True
            ln = this_row['LAST']
            mod_row['LAST'] = ln[:1] + 'z' + ln[1+1:]


        # debugging
        compare_string = "*".join([this_row['Id'], this_row['FIRST'], this_row['LAST'], this_row['ZIP'], 
            mod_row['FIRST'], mod_row['LAST']])
        
        if lmod:
            zips_ldiff.append(this_row['ZIP'])
            print("Patient diff last, same zip " +  compare_string)
        else:
            print("Patient same last, same zip " +  compare_string)

        new_rows.append(mod_row)
        
        # add some more folks with same last name
        MATE_NAMES = ['Bzob', 'Mzike', 'Jzoe', 'Jzames', 'Czlyde', 'Azthony', 'Fzord']
        num_mates = random.randint(2, 5)
        for n in range(0, num_mates):
            mate_row = mod_row.copy()
            mate_row['Id'] = "y" + str(n) + id
            mate_row['FIRST'] = MATE_NAMES[n]
            mate_row['SSN'] = ''
            mate_row['BIRTHDATE'] = ''
            mate_row['DEATHDATE'] = ''
            mate_row['MAIDEN'] = ''
            new_rows.append(mate_row)
    
new_patients_df = pd.DataFrame(new_rows)
new_patients_df.to_csv('patients/source/generated/newpatients.csv', index=False)


### Will use patient set expanded with duplicates in place of the one downloaded from synthea.

In [ ]:
%%bash

mv patients/source/generated/newpatients.csv  patients/source/downloads/10k_synthea_covid19_csv/patients.csv

### Convert synthea data for Neptune bulk load formats: both LPG and RDF.

In [ ]:
NS = "http://example.org/patientgraph"
LINKS = ["ORGANIZATION", "PATIENT", "PROVIDER", "PAYER", "ENCOUNTER"]

edge_file = csv_open("patients/bulk/pg/edges/edges.csv", ["~from","~to","~id","~label"])

def set_props(df, nep_headers, skip_links):
    l = len(df.columns)
    for i in range(l):
        col_name = df.columns[i]
        col_type = df.dtypes[i]
        if not(col_name in LINKS) or skip_links == False: 
            if col_type == 'float64':
                nep_headers.append(f"{col_name}:double")
            elif col_type == 'int64':
                nep_headers.append(f"{col_name}:long")
            else:
                nep_headers.append(f"{col_name}:string")

def make_persource_node_headers(df):
    nep_headers = ['~id', '~label']
    set_props(df, nep_headers, False)
    return nep_headers

def make_persource_edge_headers(df):
    nep_headers = ["~from","~to","~id"]
    set_props(df, nep_headers, True)
    nep_headers.append("~label")
    return nep_headers

def make_loader_file(source_csv, label):
    print("load " + label)
    df = pd.read_csv(source_csv)
    if 'ZIP' in df.columns:
        convert_dict = {'ZIP': object} 
        df = df.astype(convert_dict)
    node_headers = make_persource_node_headers(df)
    node_file = csv_open(f"patients/bulk/pg/nodes/{label}.csv", node_headers)

    for index, row in df.iterrows():
        nodeid = row['Id'] if 'Id' in row else str(index)
        uri = make_uri(f"{label.lower()}/{nodeid}")
        myrow = [str(uri), label]
        for c in df.columns:
            myrow.append(row[c])
            if c in LINKS:
            
                # special handling - encounter/patient reverse direction
                linkuri = make_uri(f"{c.lower()}/{row[c]}")
                source_node = uri
                target_node = linkuri
                link_label = c
                if label=='Encounter' and c=='PATIENT':
                    source_node = linkuri
                    target_node = uri
                    link_label = "ENCOUNTER"
                edgeuri = make_uri(f"edge-{source_node}-{target_node}")
                csv_write(edge_file, [str(source_node), str(target_node), edgeuri, f"has{link_label}"])
            
        csv_write(node_file, myrow)
    csv_close(node_file)

def make_loader_file_pt(source_csv):
    print("load pt")
    df = pd.read_csv(source_csv)
    if 'ZIP' in df.columns:
        convert_dict = {'ZIP': object} 
        df = df.astype(convert_dict)
    edge_headers = make_persource_edge_headers(df)
    ptedge_file = csv_open(f"patients/bulk/pg/edges/patient_payer_edges.csv", edge_headers)

    for index, row in df.iterrows():
        patient_uri = make_uri(f"patient/{row['PATIENT']}")
        payer_uri = make_uri(f"payer/{row['PAYER']}")
        edge_uri = make_uri(f"edge-{patient_uri}-{payer_uri}-{str(index)}")
        myrow = [str(patient_uri), str(payer_uri), str(edge_uri)]
        for c in df.columns:
            if not(c in LINKS):
                myrow.append(row[c])
        myrow.append("hasPAYER")
        csv_write(ptedge_file, myrow)
    csv_close(ptedge_file)

make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/patients.csv", 'Patient')
make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/organizations.csv", 'Organization')
make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/providers.csv", 'Provider')
make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/payers.csv", 'Payer')
make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/encounters.csv", 'Encounter')
make_loader_file_pt("patients/source/downloads/10k_synthea_covid19_csv/payer_transitions.csv")

# The remainder are interesting to link from encounter. To save space, we'll leave them. 
#make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/allergies.csv", 'Allergy')
#make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/conditions.csv", 'Condition')
#make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/medications.csv", 'Medication')
#make_loader_file("patients/source/downloads/10k_synthea_covid19_csv/careplans.csv", 'Careplan')

csv_close(edge_file)


In [ ]:
%%bash

rm patients.zip
zip patients.zip -r patients/bulk/pg

## Publications Data
DBLP/ACM dataset, known to have lots of matches.

Transform the Neptune CSV:
- Authors
- Publications - with embeddings of title added
- Edges: hasAuthor from Publication to Author


### Setup text-to-vector mapping
For use in vector similarity example
Will use all-MiniLM-L6-v2 from HuggingFace

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(sentences):
    embeddings = model.encode(sentences)
    return embeddings
    
def get_str_embedding(embedding):
    return ';'.join([str(x) for x in embedding])
    

### Create folder structure for publications data

In [ ]:
%%bash

rm -rf publications
mkdir -p publications/source/downloads
mkdir -p publications/dedup/prep/pg/nodes
mkdir -p publications/dedup/prep/pg/edges
mkdir -p publications/dedup/prep/rdf
mkdir -p publications/dedup/bulk/pg/nodes
mkdir -p publications/dedup/bulk/pg/edges
mkdir -p publications/dedup/bulk/rdf
mkdir -p publications/vss/bulk


### Download publication source dataset

In [ ]:
%%bash

cd publications/source/downloads
aws s3 cp s3://ml-transforms-public-datasets-us-east-1/dblp-acm/records/dblp_acm_records.csv .

cd ..
aws s3 cp s3://aws-neptune-customer-samples/neptune-ent-res/publications/dedup/source/dblp_acm_matches_ordered.csv dblp_acm_matches_ordered.csv 

### Add a few interesting fakes

In [ ]:
%%bash

# ADD A FEW INTERESTING FAKES

cd publications/source/downloads
echo journals/haveyweb/1,Design and findings and next steps for Java implementation of Telegraph Dataflow,Michael Havey,HaveyWeb,2023,DBLP >> dblp_acm_records.csv
echo journals/haveyweb/2,Tetanus cases increase Emerg units in September,Michael Havey,HaveyWeb,2023,DBLP >> dblp_acm_records.csv
echo journals/haveyweb/3,ER staff reports lockjaw diagnoses rose 5 percent after Labor Day,Michael Havey,HaveyWeb,2023,DBLP >> dblp_acm_records.csv


### Take initial pass through publications. No embeddings or matches yet.

In [ ]:
'''
Create property graph and RDF representations of publications list. 
In this cell, start by separating into 3 CSVs plus one ntriples
'''

NS = "http://example.org/pubgraph"

pub_file = csv_open("publications/dedup/prep/pg/nodes/pub.csv", ["~id","~label",
    "sourceid:string", "source:string", "venue:string", "year:integer", "title:string"])
aut_file = csv_open("publications/dedup/bulk/pg/nodes/author.csv", ["~id","~label","name:string"])
pa_edge_file = csv_open("publications/dedup/bulk/pg/edges/paedge.csv", ["~from","~to","~id","~label"])
rdf_file = rdf_open("publications/dedup/bulk/rdf/pubs.nt")

distinct_authors={}
df = pd.read_csv('publications/source/downloads/dblp_acm_records.csv')
for index, row in df.iterrows():
    source_id = row['id']
    source_id_esc = urllib.parse.quote(source_id)
    venue = row['venue']
    year = row['year']
    source = row['source']
    title = row['title']
    pub_uri = make_uri(f"{source}/{source_id_esc}")
    csv_write(pub_file, [str(pub_uri), "Publication" ,source_id, source, venue, year, title])
    rdf_write(rdf_file, pub_uri, RDF.type, make_uri("Publication"))
    rdf_write(rdf_file, pub_uri, make_uri("hasSource"), make_uri(source))
    rdf_write(rdf_file, pub_uri, make_uri("sourceID"), Literal(source_id))
    rdf_write(rdf_file, pub_uri, make_uri("venue"), Literal(venue))
    rdf_write(rdf_file, pub_uri, make_uri("year"), Literal(year))
    rdf_write(rdf_file, pub_uri, make_uri("title"), Literal(title))

    if not pd.isna(row['authors']):
        auts = row['authors'].split(",")
        for a in auts:
            sa = a.strip()
            sau = urllib.parse.quote(sa)
            author_uri=make_uri(f"author/{sau}")
            edge_uri =  make_uri(f"hasAuthor/{source}/{source_id}/{author_uri}")
            csv_write(pa_edge_file, [str(pub_uri), str(author_uri), str(edge_uri), "hasAuthor"])
            rdf_write(rdf_file, pub_uri, make_uri("hasAuthor"), author_uri)
            if not(sa in distinct_authors):
                csv_write(aut_file, [str(author_uri), "Author", sa])
                rdf_write(rdf_file, author_uri, RDF.type, make_uri("Author"))
                rdf_write(rdf_file, author_uri, make_uri("name"), Literal(sa))
                distinct_authors[sa] = "a"

csv_close(pub_file)
csv_close(aut_file)
csv_close(pa_edge_file)
rdf_close(rdf_file)


### Add embeddings to publication. This is for VSS demo only.

In [ ]:
'''
For VSS demo, write publications with embeddings.
'''

pub_file = csv_open("publications/vss/bulk/pub.csv", ["~id","~label",
    "sourceid:string", "source:string", "venue:string", "year:integer", "title:string", "embedding:vector"])

iter = 1
df = pd.read_csv('publications/dedup/prep/pg/nodes/pub.csv')
for index, row in df.iterrows():
    iter += 1
    if iter % 100 == 0:
        print(str(iter))
    myrow = [
        row["~id"],
        row["~label"], 
        row["sourceid:string"], 
        row["source:string"], 
        row["venue:string"], 
        row["year:integer"], 
        row["title:string"], 
        get_str_embedding(get_embeddings(row["title:string"]))
    ]
    csv_write(pub_file, myrow)

csv_close(pub_file)

### Build dedup match representation. This is for the dedup demo only. This demo is not discussed in the blog post.

In [ ]:
'''
For dedup exploration:
- Two new labels for a publication: AuthoritativePublication, LinkedPublication
- New relationship: matches

We don't need embeddings here.

As input, dblp_acm_matches_ordered.csv is sorted by match id. Within a match group most recent DBLP comes first
and is considered authoritative

'''

pub_file = csv_open("publications/dedup/bulk/pg/nodes/pub.csv", ["~id","~label",
    "sourceid:string", "source:string", "venue:string", "year:integer", "title:string"])
pp_edge_file = csv_open("publications/dedup/bulk/pg/edges/ppedge.csv", ["~from","~to","~id", 
    "matchSource:string", "matchAlg:string", "matchReason:string", "~label"])
rdf_file = rdf_open("publications/dedup/bulk/rdf/matches.nt")


'''
Build dictionary from match file
'''
pub_match_dict = {}
match_df = pd.read_csv('publications/source/dblp_acm_matches_ordered.csv')
last_match_id = ""
last_lead_id = ""
last_lead_source = ""
for index, row in match_df.iterrows():
    source_id = row['id']
    match_id = row['match_id']
    source=row['source']
    if match_id == last_match_id:
        lead_id_esc = urllib.parse.quote(last_lead_id)
        lead_uri = make_uri(f"{last_lead_source}/{lead_id_esc}")
        pub_match_dict[source_id] = {
            'alt_label': 'LinkedPublication', 
            'group': match_id,
            'lead': lead_uri
        }
    else:
        last_lead_id = source_id
        last_match_id = match_id
        last_lead_source = source
        pub_match_dict[source_id] = {
            'alt_label': 'AuthoritativePublication'
        }

def get_match_detail(source_id):
    if source_id in pub_match_dict:
        return pub_match_dict[source_id]
    else:
        return {
            'alt_label': 'AuthoritativePublication'
        }
        

'''
Rebuild publications based on matchs
'''

pub_df = pd.read_csv('publications/dedup/prep/pg/nodes/pub.csv')
for index, row in pub_df.iterrows():
    myrow = [
        row["~id"],
        row["~label"], 
        row["sourceid:string"], 
        row["source:string"], 
        row["venue:string"], 
        row["year:integer"], 
        row["title:string"]
    ]
    match_detail = get_match_detail(myrow[2])
    alt_label = match_detail['alt_label']
    myrow[1] = f"{myrow[1]};{alt_label}"
    csv_write(pub_file, myrow)
    pub_uri = URIRef(myrow[0])
    rdf_write(rdf_file, pub_uri, RDF.type, make_uri(alt_label)) 
    
    if 'lead' in match_detail:
        match_source="dedup"
        match_alg="FindMatches"
        match_reason=match_detail['group']
        lead_uri = match_detail['lead']
        edge_uri = make_uri(f"matches-{str(pub_uri)}-{str(lead_uri)}")
        csv_write(pp_edge_file, [lead_uri, myrow[0], edge_uri, match_source, match_alg, match_reason, "matches"])

        # Avoid the bnode here to make ingest repeatable
        match_uri = make_uri(f"match-{row['sourceid:string']}")
        rdf_write(rdf_file, pub_uri, make_uri("hasMatch"), match_uri)
        rdf_write(rdf_file, match_uri, make_uri("hasLead"), lead_uri)
        rdf_write(rdf_file, match_uri, make_uri("hasMatchSource"), make_uri(match_source))
        rdf_write(rdf_file, match_uri, make_uri("hasMatchAlgorithm"), make_uri(match_alg))
        rdf_write(rdf_file, match_uri, make_uri("matchReason"), Literal(match_reason))
        
csv_close(pub_file)
csv_close(pp_edge_file)
rdf_close(rdf_file)